In [1]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime
import os


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',20)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

In [4]:


# Función para normalizar texto
def normalize_text(text):
    # Reemplazar caracteres especiales y convertir a minúsculas para una comparación consistente
    text = text.replace("/", " ")  # Reemplazar la barra por espacio o eliminarla completamente
    text = text.replace("ñ", "n")  # Reemplazar la "ñ" por "n"
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [5]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
    
    return df


In [6]:
def porcentaje_nulos(df, columnas):
    # Diccionario para almacenar los resultados por columna
    resultados = {}
    
    # Calcular el porcentaje de nulos para cada columna
    for columna in columnas:
        total_filas = len(df)
        nulos = df[columna].isnull().sum()
        porcentaje_nulos = (nulos / total_filas) * 100
        resultados[columna] = porcentaje_nulos
    
    return resultados



# Carga de datos

In [3]:

print(os.getcwd())

c:\Users\extas\Documents\GitHub\precios-de-vivienda-madrid\codigo\01-preprocesamiento-de-datos\Revision_columnas


In [4]:
df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_21308\2425994981.py:1: DtypeWarning: Columns (5,7,16,28,32,70,71,76,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)


In [23]:
df17 = df17.loc[:, ~df17.columns.str.contains('^Unnamed')]

In [21]:
df17.shape

(10529, 113)

In [24]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [25]:
df17['cantidad_visitas'] = np.where((df17['cantidad_visitas'].isnull()) | (df17['cantidad_visitas'] == 0), df17['n.visitas'], df17['cantidad_visitas'])


In [26]:
df17['n.visitas'].value_counts(dropna=False)

n.visitas
NaN       9647
30.00      153
40.00      139
20.00      130
50.00       97
          ... 
170.00       2
140.00       1
200.00       1
190.00       1
180.00       1
Name: count, Length: 21, dtype: int64

In [20]:
# Guardar el DataFrame en un archivo CSV sin incluir el índice
#df.to_csv("dfpisosCompleto17012025.csv", index=False)


In [29]:
df17["porteria"].value_counts(dropna=False)

porteria
NaN                      10372
Portero todo el día        153
Portero media jornada        4
Name: count, dtype: int64

# revision columnas

In [64]:


# Supongamos que df17 es tu DataFrame
#df17['porteria'] = df17['porteria'].replace('0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('0.0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('Climatización', 'climatizacion')
df17['calefaccion'] = df17['calefaccion'].replace('Climatizaciòn', 'climatizacion')
df17['tiene_armario'] = df17['tiene_armario'].replace(0.0, np.nan)
df17.loc[df17['web'].isnull(), 'web'] = 'Indomio.com'
df17['tiene_piscina'] = df17['tiene_piscina'].replace(0, np.nan)


df17['porteria'] = df17['porteria'].replace('1', 'Portero todo el día')


## calefaccion

In [106]:
df17["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                                                   6949
Central                                                927
Individual                                             646
1.0                                                    578
Gas natural                                            520
Calefaccion Gas Natural                                256
Calefaccion Electrica                                  126
Eléctrica                                              113
Calefaccion Individual                                 104
Calefaccion Central                                     87
Individual, alimentación eléctrica                      34
Individual, por radiadores, de gas                      31
Individual - gas natural                                18
Central, por radiadores, de gas                         16
Suelo radiante                                          11
Individual, por radiadores                              10
Gasoil                                      

In [138]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto
    
 
    
    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)
    
    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)
      
   
    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)
    
   
    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)    
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else: 
        radiadores.add(categoria)
    

# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Central, por radiadores', 'Central, por radiadores, de gas', 'Comunitaria', 'Comunitario', 'Central individualizada', 'Agua caliente y calefacción central con contador i', 'Calefacción central individualizada', 'Central, suelo radiante', 'Central, por aire, de gas', 'Centralizada', 'Central, de gasóil', 'Central', 'Central, alimentación eléctrica', 'Central, suelo radiante, de gas', 'Central, por radiadores, de gasóil', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas']
Individual: ['Individual, suelo radiante, de gas', 'Bomba de calor', 'climatizacion', 'Bomba frio-calor', 'Gas particular', 'Individual, por radiadores', 'Individual con gas natural', 'Bomba frio calor', 'Individual, de gasóil', 'Individual, por radiadores, alimentación eléctrica', 'Calefaccion Individual', 'Calefaccion individual', 'Contadores individuales', 'Individual, suelo radiante', 'Individual, alimentación eléctrica', 'Individual - gas natural', 'Individual', 'Climatizacion', 'Indivi

In [7]:
df17["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN                     6994
Más de 50 años          1018
Entre 30 y 50 años       205
 Mas de 50 anos          131
1900.0                   118
                        ... 
 Entre 10 y  15 Anos       1
 1949                      1
 1943                      1
 1928                      1
1944.0                     1
Name: count, Length: 188, dtype: int64

In [9]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10295
6.00        45
4.00        42
3.00        32
7.00        25
5.00        24
8.00        21
2.00        14
10.00       14
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

In [11]:
porcentaje = porcentaje_nulos(df17, 'plantas_edificio')
print(f"Porcentaje de valores nulos en 'plantas_edificio': {porcentaje:.2f}%")

Porcentaje de valores nulos en 'plantas_edificio': 97.78%


In [ ]:
df17["plantas_edificio"].value_counts(dropna=False)

## aire acondicionado

In [12]:
df17["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN                                                 5976
0                                                   1038
0.0                                                  845
Individual                                           645
Frío                                                 480
                                                    ... 
Aire frio/ calor por conductos/suelo refrescante       1
No disponible                                          1
Centralizada, frío                                     1
Por conductos en toda la casa                          1
Climatizacion por conductos frio y calor               1
Name: count, Length: 36, dtype: int64

In [13]:
df17["aire_acondicionado"].unique()


array([nan, 'Preinstalación', 'Individual, frío/calor',
       'Individual, frío', 'Individual', 'Frío/calor',
       'Centralizada, frío', 'Centralizada, frío/calor',
       'Centralizada, calor', 'Centralizada', 'Frío', 'Frío y calor',
       'Splites repartidos', 'Por conductos en toda la casa',
       'Aerotermia', 'Independiente', 'Con bomba de calor', 'A/a f/c',
       'Suelo refrigerante', 'Preinstalado', 'Frío-calor',
       'Suelo radiante refrescante', 'Sistema air zone',
       'Climatizacion por conductos frio y calor',
       'Frió/calor por conductos', 'Central', 'Suelo radiante',
       'Tiene la instalción hecha en toda la casa por cond',
       'Geotermia y aerotermia', 'En toda la casa, por conductos',
       'Aire frio/ calor por conductos/suelo refrescante',
       'No disponible', '1', '0', '0.0', '1.0'], dtype=object)

In [ ]:
# Lista de valores para cada grupo
eficientes = [
    'Aerotermia', 'Geotermia y aerotermia', 'Suelo radiante refrescante', 'Suelo refrigerante',
    'Sistema air zone', 'Climatizacion por conductos frio y calor', 'Frió/calor por conductos', 
    'Suelo radiante', 'Aire frio/ calor por conductos/suelo refrescante','Por conductos en toda la casa','Tiene la instalción hecha en toda la casa por cond','En toda la casa, por conductos'
]

normales = [
    'Individual, frío/calor', 'Individual, frío', 'Individual', 'Frío/calor', 'Centralizada, frío', 
    'Centralizada, frío/calor', 'Centralizada, calor', 'Central', 'Frío', 'Frío y calor', 
    'Splites repartidos',  'Independiente', 'Con bomba de calor','A/a f/c', 
    'Aire frio/calor por conductos/suelo refrescante','1','1.0'
]

Sin_aire = ['Preinstalación', 'Preinstalado','No disponible',  '0', '0.0']

# Función para clasificar los sistemas
def clasificar_sistemas(df, columna):
    modernos = []
    normales = []
    Sin_aire = []
    
    for valor in df[columna]:
        if valor in modernos_y_eficientes:
            modernos.append(valor)
        elif valor in normales:
            normales.append(valor)
        elif valor in sin_aire:
           Sin_aire.append(valor)
    
    return modernos, normales, sin_info

# Llamar a la función con el DataFrame y la columna deseada
modernos, normales, sin_info = clasificar_sistemas(df17, "aire_acondicionado")

# Mostrar resultados
print("Sistemas modernos y eficientes:", modernos)
print("Sistemas normales:", normales)
print("Sin información relevante:", sin_info)


In [14]:
df17["jardin"].unique()

array([nan, 'Privado', 'Sin jardín', 'Comunitario', '0.0', '1.0'],
      dtype=object)

In [15]:
porcentaje = porcentaje_nulos(df17, 'jardin')
print(f"Porcentaje de valores nulos en jardin': {porcentaje:.2f}%")

Porcentaje de valores nulos en jardin': 78.93%


In [24]:

# Ejemplo de uso
columnas = ["inmueble_ingresos", "alquiler_opcion_a_compra", "disponibilidad","nuda_propiedad","sup_comercial"]  # Reemplaza con las columnas de tu DataFrame
porcentaje_resultados = porcentaje_nulos(df17, columnas)

# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en inmueble_ingresos: 99.92%
Porcentaje de nulos en alquiler_opcion_a_compra: 99.97%
Porcentaje de nulos en disponibilidad: 95.55%
Porcentaje de nulos en nuda_propiedad: 99.98%
Porcentaje de nulos en sup_comercial: 96.46%


In [19]:
df17["disponibilidad"].value_counts(dropna=False)

disponibilidad
NaN      10060
Libre      469
Name: count, dtype: int64

La columna disponibilidad no aporta nada

In [23]:
df17["alquiler_opcion_a_compra"].value_counts(dropna=False)

alquiler_opcion_a_compra
NaN     10526
1.00        3
Name: count, dtype: int64

In [32]:
df17.query("(inmueble_ingresos == 1) | (alquiler_opcion_a_compra == 1) | (nuda_propiedad == 1)").count()


title                    13
url                      13
precio                   13
precio_anterior           0
descuento                 0
                         ..
orientacion_oeste         0
orientacion_sur           2
orientacion_sur,_este     0
sistema_alarma            2
n.visitas                 0
Length: 111, dtype: int64

## sup_comercial

In [33]:
df17[df17['sup_comercial'].notnull()].head()

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,cocina_equipada,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,cocina_1,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas
2,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",https://www.indomio.es/anuncios/81008864/,200000.00,NaN,NaN,500.00,400.00,Arganzuela,2.00,Legazpi,24.00,Legazpi,plaza de italia,Casa plurifamiliar Plaza de Italia,desconocido,NaN,No indicado,Casa,NaN,NaN,NaN,Sótano (-5),NaN,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,23.00,NaN,Privado,NaN,NaN,NaN,NaN,401.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Escribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorioEscribe una descripción de al menos 16 palabras obligatorio,NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,1.00,NaN,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,"Piso de tres habitaciones buen estado, Ensanche de Vallecas-Valdecarros, Madrid",https://www.indomio.es/anuncios/87841991/,450000.00,NaN,NaN,4.59,98.00,Villa de Vallecas,18.00,EnsanchedeVallecas,183.00,Ensanche de Vallecas,NaN,Piso de tres habitaciones buen estado,desconocido,NaN,No indicado,Piso,NaN,Bueno / Habitable,No,1,3,3.00,2,1.00,NaN,1 en aparcamiento/garaje público,1.00,1.00,NaN,Individual,2010.0,NaN,NaN,Sin jardín,NaN,NaN,NaN,NaN,203.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,"Bonito piso en el Ensanche, cerca del metro y todos los sservicios. Cerca del colegio Stella Maris, metro la gavia y cc.la gavia. Bien comunicado con la A3. Buen estado de conservación.",NaN,NaN,NaN,Indomio.com,2024-11-20,No indicado,-,0.00,NaN,183.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,"Piso de dos habitaciones buen estado, San Diego, Madrid",https://www.indomio.es/anuncios/88009171/,210000.00,NaN,NaN,2.80,75.00,Puente de Vallecas,13.00,SanDiego,132.00,San Diego,NaN,Piso de dos habitaciones buen estado,desconocido,B,No in

In [30]:
df17['tipo_inmueble'].value_counts(dropna=False)

tipo_inmueble
NaN          9959
económico     276
medio         173
señorial      121
Name: count, dtype: int64

In [40]:
df17['tiene_armario'].value_counts(dropna=False)

tiene_armario
NaN     9811
1.00     718
Name: count, dtype: int64

In [43]:
df17['cancha_tenis'].value_counts(dropna=False)

cancha_tenis
NaN     7091
0.00    3435
1.00       3
Name: count, dtype: int64

In [44]:
df17['chimenea'].value_counts(dropna=False)

chimenea
NaN     6209
0.00    4268
1.00      52
Name: count, dtype: int64

In [45]:
df17[df17['chimenea'] == 1][['calefaccion', 'chimenea']]

,calefaccion,chimenea
22,NaN,1.00
81,NaN,1.00
208,"Central, por radiadores, de gas",1.00
212,"Individual, por radiadores, de gas",1.00
227,NaN,1.00
...,...,...
9581,Individual,1.00
9746,1.0,1.00
9749,1.0,1.00
9750,1.0,1.00


In [46]:
df17['fibra_optica'].value_counts(dropna=False)

fibra_optica
NaN     7091
0.00    3396
1.00      42
Name: count, dtype: int64

In [66]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     10097
1.00      432
Name: count, dtype: int64

In [67]:
df17['carpinteria_exterior_doble_vidrio/pvc'].value_counts(dropna=False)

carpinteria_exterior_doble_vidrio/pvc
NaN     7091
0.00    3419
1.00      19
Name: count, dtype: int64

In [ ]:
df17[df17['tipo_inmueble'].notnull()]

In [73]:
df17['cocina'].value_counts(dropna=False)

cocina
NaN                                   9500
Cocina independiente                   323
Cocina amueblada                       190
Independiente                          131
Cocina abierta                          72
                                      ... 
Francesa                                 1
Equipada con excelentes calidades        1
Con entrada independiente                1
Independiente amueblada y equipada       1
Amueblada reformada                      1
Name: count, Length: 87, dtype: int64

In [74]:
df17['cocina'].unique()

array([nan, 'Cocina abierta', 'Cocina independiente', 'Cocina americana',
       'Amueblada con electrodomésticos', 'Cocina equipada', 'Americana',
       'Independiente y equipada', 'Amueblada sin electrodomésticos',
       'Cocina amueblada', 'Independiente', 'Amueblada, equipada',
       'Individual', 'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',
       'Co oficce, dormitorio de servicio y entrada indep.',
       'Independiente con des

In [75]:



# Define las categorías
def agrupar_cocinas(tipo):
    categorias = []
    if pd.isnull(tipo):
        categorias.append('Sin_información')
    if 'abierta' in tipo.lower() or 'abierto' in tipo.lower()  or'americana' in tipo.lower() or 'integrada' in tipo.lower():
        categorias.append('Abierta')
    if 'independiente' in tipo.lower():
        categorias.append('Independiente')
    if 'amplia' in tipo.lower():
        categorias.append('amplia')
    if 'isla' in tipo.lower():
        categorias.append('con isla')
    if 'amueblada' in tipo.lower() 
        categorias.append('Amueblada')
    if 'con electrodomesticos' in tipo.lower() or 'equipada' in tipo.lower():
        categorias.append('Equipada con electrodomésticos')
    if 'sin amueblar' in tipo.lower():
        categorias.append('No amueblada')   
    if 'reformada' in tipo.lower():                 
         return 'Reformada'
    elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
          return 'Sin reforma'
    if not categorias:
        categorias.append('Sin reforma')
    return ', '.join(categorias)



In [ ]:
# Aplica la función de agrupación
df17['grupo_cocina'] = df17['cocina'].apply(agrupar_cocinas)

print(df17[['cocina', 'grupo_cocina']])


In [ ]:

# Define las categorías
def agrupar_cocinas(df,columna):
    # Clasificar categorías
    for categoria in df1[columna]:
        tipo = normalize_text(str(categoria))  # Normalizar texto
        if pd.isnull(tipo):
          return 'Sin información'
        elif 'abierta' in tipo.lower() or 'americana' in tipo.lower() or 'integrada' in tipo.lower():
         return 'Abierta'
        elif 'independiente' in tipo.lower():
         return 'Independiente'
        elif 'amueblada' in tipo.lower() or 'equipada' in tipo.lower():
         return 'Amueblada'
        elif 'sin amueblar' in tipo.lower():
         return 'No amueblada'
        elif 'reformada' in tipo.lower():                 
         return 'Reformada'
        elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
          return 'Sin reforma'
        else:
          return 'sin clasificar'



In [ ]:
# Aplica la función de agrupación
df17['grupo_cocina'] = agrupar_cocinas(df17,'cocina')

print(df17[['tipo_cocina', 'grupo_cocina']])


In [69]:
df17['cocina_1'].value_counts(dropna=False)

cocina_1
NaN                     10522
Cocina independiente        5
Cocina abierta              2
Name: count, dtype: int64

In [70]:
df17['cocina_equipada'].value_counts(dropna=False)

cocina_equipada
NaN     9059
0.00     904
1.00     566
Name: count, dtype: int64

In [71]:
df17['amueblado'].value_counts(dropna=False)

amueblado
NaN                       7606
0                         1469
0.0                        721
Sí                         305
No                         175
1.0                        161
Sólo cocina amueblada       78
Parcialmente amueblado      13
1                            1
Name: count, dtype: int64

In [42]:
df17.to_csv("dfpisosCompleto17012025.csv",index='False')

In [32]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [46]:
# Contar las filas donde 'dormitorios' es igual a 'cantidad_dormitorios'
conteo_filas_iguales = (df17['dormitorios'] == df17['cantidad_dormitorios']).sum()

print(conteo_filas_iguales)


22


In [29]:
df17["cod_barrio"].value_conts(dropna=False)

AttributeError: 'Series' object has no attribute 'value_conts'